In [1]:
def notNaN(num):
    return num == num
def isNaN(num):
    return num != num

def word(f, w):
    f.write("{\"lower\":\"" + w + "\"}")

In [2]:
import pandas as pd

def label2jsonl(label):
    doc = pd.read_csv("../dataset/labels/"+ label + ".csv", low_memory=False)
    f = open("../dataset/Label2Train/labels.jsonl","a")

    i = 0
    for var in doc["variation"]:
        if(notNaN(doc["RC"][i]) and doc["RC"][i]!="xxx"):
            f.write("{\"label\": \"" + label+"\", \"pattern\":[")
            term = doc["RC"][i].split(' ')
            for j, w in enumerate(term):
                if j < len(term) - 1:
                    word(f, w)
                    f.write(",")
                else:
                    word(f, w)
            f.write("]}\n")
        elif(isNaN(doc["RC"][i])):
            f.write("{\"label\": \"" + label+"\", \"pattern\":[")
            term = str(var).split(' ')
            for j, w in enumerate(term):
                if j < len(term) - 1:
                    word(f, w)
                    f.write(",")
                else:
                    word(f, w)
            f.write("]}\n")      
        i+=1
    f.close()
    


In [3]:
labels = ["nanoparticle", "property", "material", "event", "technique"]
for label in labels:
    label2jsonl(label)


FileNotFoundError: [Errno 2] No such file or directory: '../dataset/labels/nanoparticle.csv'

In [4]:

import pandas as pd
def listTerms(term):
    l = []
    for i in term:
        if(notNaN(i)):
            l.append(i)
    return l

import random

def remove_random_elements(input_list, percentage=0.1):
    removed_elements = []
    num_elements_to_remove = int(len(input_list) * percentage)
    
    for _ in range(num_elements_to_remove):
        index = random.randint(0, len(input_list) - 1)
        removed_element = input_list.pop(index)
        removed_elements.append(removed_element)
    
    return input_list, removed_elements

def abLabel2jsonl(label):
    percentage = 0.1
    doc = pd.read_csv("../dataset/labels/"+ label + ".csv", low_memory=False)
    f = open("../dataset/Label2Train/labels_abreaction.jsonl","a")

    lterm = listTerms(doc["terms"])
    lterm, removed_elements = remove_random_elements(lterm, percentage)
    i = 0
    t=""
    for var in doc["variation"]: 
        if(notNaN(doc["terms"][i])):
            t = doc["terms"][i]  
        if(t in lterm):
            if(notNaN(doc["RC"][i]) and doc["RC"][i]!="xxx"): 
                f.write("{\"label\": \"" + label+"\", \"pattern\":[")
                term = doc["RC"][i].split(' ')
                for j, w in enumerate(term):
                    if j < len(term) - 1:
                        word(f, w)
                        f.write(",")
                    else:
                        word(f, w)
                f.write("]}\n")
            elif(isNaN(doc["RC"][i])):
                f.write("{\"label\": \"" + label+"\", \"pattern\":[")
                term = str(var).split(' ')
                for j, w in enumerate(term):
                    if j < len(term) - 1:
                        word(f, w)
                        f.write(",")
                    else:
                        word(f, w)
                f.write("]}\n")      
        i+=1
    f.close()
    


In [5]:
import math
doc = pd.read_csv("../dataset/labels/"+ "nanoparticle" + ".csv", low_memory=False)

print()

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/labels/nanoparticle.csv'

In [45]:
labels = ["nanoparticle", "property", "material", "event", "technique"]
for label in labels:
    abLabel2jsonl(label)

In [5]:
import pandas as pd
def listTerms(term):
    l = []
    for i in term:
        if(notNaN(i)):
            l.append(i)
    return l
import random

def remove_random_elements(input_list, percentage=0.1):
    removed_elements = []
    num_elements_to_remove = int(len(input_list) * percentage)
    
    for _ in range(num_elements_to_remove):
        index = random.randint(0, len(input_list) - 1)
        removed_element = input_list.pop(index)
        removed_elements.append(removed_element)
    
    return input_list, removed_elements
def listAbre(label, lvar, ldel, percentage=0.1):
    doc = pd.read_csv("../database/corpus//labels/"+ label + ".csv", low_memory=False)
    # f = open("../dataset/Label2Train/labels_abreaction.jsonl","a")

    lterm = listTerms(doc["terms"])
    lterm, removed_elements = remove_random_elements(lterm, percentage)
    i = 0
    t=""
    for var in doc["variation"]: 
        if(notNaN(doc["terms"][i])):
            t = doc["terms"][i]  
        if(t in lterm):
            if(notNaN(doc["RC"][i]) and doc["RC"][i]!="xxx"): 
                term = doc["RC"][i]
            elif(isNaN(doc["RC"][i])):
                term = str(var)
            lvar.append(term)
        else:
            ldel.append(var)
            
        i+=1
    return lvar, ldel


In [26]:
labels = ["nanoparticle", "property", "material", "event", "technique"]
lvar=[]
ldel=[]
n=5
fdel_path = f"./../database/ablation/{n}/listDelete{n}.txt"
fvar_path = f"./../database/ablation/{n}/listvar{n}.txt"
for label in labels:
    lvar, ldel = listAbre(label, lvar, ldel, percentage=0.2)
    with open(fdel_path,"w") as fdel:
        with open(fvar_path,"w") as fvar:
            for i in lvar:
                fvar.writelines(f"{i}\n")
            for i in ldel:
                fdel.writelines(f"{i}\n")


In [27]:
import json
import re
input_file = f'./../database/corpus/preparation/train_all.txt'
output_eval = f'./../database/ablation/{n}/eval.txt'
output_train = f'./../database/ablation/{n}/train_abla.txt'


with open(input_file, 'r') as f:
    data = [json.loads(line) for line in f]

feval = open(output_eval, "w")
ftrain = open(output_train, "w")
t = e = 0
for example in data:
    text = example['text']
    entities = example['spans']
    
    words = text.split(' ')

    tokens = [re.findall(r'[)|(|,|.]+|\b\w+\b|-', word) for word in words]

    words = [token for sublist in tokens for token in sublist if token != '']

    # print(result)
    # words = re.split('([-])', text)
    labels = ['O'] * len(words)
    train = 1
    for ent in entities:
        start = ent['start']
        end = ent['end']
        label = ent['label']
        count= 0
        if(text[start:end] in ldel):
            train = 0
        for i, word in enumerate(words):
            word_start = text.index(word, count)
            word_end = word_start + len(word)
            count += len(word)
            #print(count, text[count], words[i])
            if start <= word_start < end:
                if(start==word_start):
                    labels[i] = 'B-' + label
                    count = word_start + len(word)
                else:
                    labels[i] = 'I-' + label
    if train == 1:
        t+=1
    else :
        e+=1
    f = ftrain if train else feval
    for i in range(len(words)):
        f.write(f"{words[i]} {labels[i]}\n")
    f.write('\n')
print("num train: ", t," num eval: ", e)



num train:  116686  num eval:  31576
